In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

from scipy.interpolate import interp1d
from pqcd.utils import (
    eos_dir,
    nsat,
    to_GeV_per_cubic_femtometre, 
    to_nucleons_per_cubic_femtometre, 
    weighted_quantile,
)

In [2]:
collated_eos_path = '../data/eos-draws-default/collated_np_all_post.csv'

In [3]:
collated_eos = pd.read_csv(collated_eos_path)

astro_weights = np.exp(collated_eos.logweight_total.values)
collated_eos_nonzero_astro = collated_eos[collated_eos.logweight_total > -np.inf]

In [4]:
collated_eos

,eos,logweight_total,Mmax,pressurec2(baryon_density=2.8e+14),pressurec2(baryon_density=5.6e+14),pressurec2(baryon_density=1.68e+15),energy_densityc2(baryon_density=2.8e+14),energy_densityc2(baryon_density=5.6e+14),energy_densityc2(baryon_density=1.68e+15),R(M=1.4),...,logweight_LVC_GW190425_PhenomPNRThi,Lambda(M=1.6),Lambda(M=1.8),R(M=2.2),Lambda(M=1.0),Lambda(M=1.2),Lambda(M=2.2),R(M=1.2),R(M=1.0),logweight_Fonseca_J0740
0,2265967.0,-inf,1.700984,3.139719e+12,4.073337e+13,4.278534e+14,2.842860e+14,5.828413e+14,2.064928e+15,11.985060,...,2.679168,165.095193,0.0,6.4988,2545.214606,955.596386,0.0,11.934217,11.866349,-inf
1,264698.0,-inf,1.590114,3.624216e+11,5.617729e+11,4.585547e+14,2.800613e+14,5.603871e+14,1.760666e+15,8.100073,...,2.795757,0.000000,0.0,6.4988,274.540682,106.508567,0.0,7.860560,7.545702,-inf
2,1683176.0,-inf,0.402205,5.612847e+12,6.763479e+12,8.613412e+12,2.775434e+14,5.602483e+14,1.689050e+15,4.130000,...,2.979787,0.000000,0.0,6.4988,0.000000,0.000000,0.0,3.544800,2.954000,-inf
3,696150.0,-inf,0.860414,2.399372e+12,5.710139e+12,2.346316e+14,2.844703e+14,5.727288e+14,1.770215e+15,4.130000,...,2.867603,0.000000,0.0,6.4988,0.000000,0.000000,0.0,3.544800,2.954000,-inf
4,275937.0,-inf,1.236473,2.442829e+11,1.612589e+12,1.701714e+14,2.741824e+14,5.488558e+14,1.725925e+15,4.130000,...,2.934103,0.000000,0.0,6.4988,121.700260,24.083029,0.0,7.045639,7.337356,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309536,280847.0,-inf,0.891053,3.731226e+12,4.280452e+12,5.440772e+12,2.910248e+14,5.842226e+14,1.754907e+15,4.130000,...,2.566539,0.000000,0.0,6.4988,0.000000,0.000000,0.0,3.544800,2.954000,-inf
309537,459317.0,-inf,0.590972,3.362687e+12,4.641949e+12,9.133094e+13,2.611666e+14,5.258588e+14,1.641949e+15,4.130000,...,2.992270,0.000000,0.0,6.4988,0.000000,0.000000,0.0,3.544800,2.954000,-inf
309538,2234499.0,-inf,1.665527,3.890397e+12,4.155894e+13,3.725335e+14,2.836612e+14,5.811064e+14,2.049550e+15,12.147656,...,2.599360,115.083765,0.0,6.4988,2736.187795,1026.861257,0.0,12.390059,12.365665,-inf
309539,2285403.0,-inf,1.415895,1.298773e+12,9.396924e+12,3.111593e+14,2.825170e+14,5.678329e+14,1.811870e+15,7.935074,...,3.062617,0.000000,0.0,6.4988,278.655784,87.238253,0.0,8.280072,8.295072,-inf


In [5]:
# Download the EOSs

download = True
downloaded_eos = []

if download:

    # A sample of EOSs from the prior
    for eos in collated_eos.eos[7500:8000]:

        eos = int(eos)
        destination_dir = f'../data/eos-draws-default/mrgagn/DRAWmod1000-{int(eos/1000):06}'

        if not os.path.exists(destination_dir):
            os.mkdir(destination_dir)

        os.system(
            'scp '
            f'lg:{eos_dir}/DRAWmod1000-{int(eos/1000):06}/eos-draw-{eos:06}.csv '
            f'{destination_dir}/eos-draw-{eos:06}.csv'
        )

        os.system(
            'scp '
            f'lg:{eos_dir}/DRAWmod1000-{int(eos/1000):06}/macro-draw-{eos:06}.csv '
            f'{destination_dir}/macro-draw-{eos:06}.csv'
        )

        downloaded_eos.append(eos)

    # A sample of EOSs with non-zero astro weight
    # for eos in collated_eos_nonzero_astro.eos[:2000]:

    #     eos = int(eos)
    #     if eos in downloaded_eos:
    #         continue

    #     destination_dir = f'../data/eos-draws-default/mrgagn/DRAWmod1000-{int(eos/1000):06}'

    #     if not os.path.exists(destination_dir):
    #         os.mkdir(destination_dir)

    #     os.system(
    #         'scp '
    #         f'lg:{eos_dir}/DRAWmod1000-{int(eos/1000):06}/eos-draw-{eos:06}.csv '
    #         f'{destination_dir}/eos-draw-{eos:06}.csv'
    #     )

    #     os.system(
    #         'scp '
    #         f'lg:{eos_dir}/DRAWmod1000-{int(eos/1000):06}/macro-draw-{eos:06}.csv '
    #         f'{destination_dir}/macro-draw-{eos:06}.csv'
    #     )

### Pressure vs energy density

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')

for eos in collated_eos.eos[:2000]:

    eos = int(eos)
    df = pd.read_csv(
        f'../data/eos-draws-default/mrgagn/DRAWmod1000-{int(eos/1000):06}/eos-draw-{eos:06}.csv'
    )

    pressure = to_GeV_per_cubic_femtometre(df.pressurec2)
    energy_density = to_GeV_per_cubic_femtometre(df.energy_densityc2)

    ax.plot(
        energy_density,
        pressure,
        alpha=0.1, 
        c='C0'
        )
    
for eos in collated_eos_nonzero_astro.eos[:1200]:

    eos = int(eos)
    df = pd.read_csv(
        f'../data/eos-draws-default/mrgagn/DRAWmod1000-{int(eos/1000):06}/eos-draw-{eos:06}.csv'
    )

    pressure = to_GeV_per_cubic_femtometre(df.pressurec2)
    energy_density = to_GeV_per_cubic_femtometre(df.energy_densityc2)

    ax.plot(
        energy_density,
        pressure,
        alpha=0.1, 
        c='C1'
        )

ax.set_ylim(1e-3, 1)
ax.set_xlim(0., 2)

ax.set_xlabel('Energy density [GeV/fm$^3$]')
ax.set_ylabel('Pressure [GeV/fm$^3$]');

We can alternatively use the central density, which I think is the same as nTOV, which is like the last "physical" point of the EOS?

In [ ]:
fig, ax = plt.subplots()
ax.hist(
    to_nucleons_per_cubic_femtometre(collated_eos['rhoc(M@Mmax)'])/nsat, 
    bins=100, 
    density=True,
    weights=np.exp(collated_eos.logweight_total.values),
    )
_ = ax.set_xlabel('Central density [$n_\mathrm{sat}$]')

In [ ]:
# Note that some EOSs have nTOV greater than 35 nsat, which breaks the pQCD
# likelihood function
max(to_nucleons_per_cubic_femtometre(collated_eos['rhoc(M@Mmax)'])/nsat)

### Interpolation and quantile calculation tests

In [ ]:
# We need to evaluate all thhe EOSs on a consistent array of epsilon values
# to construct quantiles
energy_density_grid = np.linspace(1e-5, 5, 1000)

pressure_interp = []
number_density_interp = []

for eos in collated_eos.eos[:2000]:

    eos = int(eos)

    df = pd.read_csv(
        f'../data/eos-draws-default/mrgagn/DRAWmod1000-{int(eos/1000):06}/eos-draw-{eos:06}.csv'
        )

    pressure = to_GeV_per_cubic_femtometre(df.pressurec2).values
    energy_density = to_GeV_per_cubic_femtometre(df.energy_densityc2).values
    number_density = to_nucleons_per_cubic_femtometre(df.baryon_density).values

    pressure_interp.append(interp1d(energy_density, pressure)(energy_density_grid))
    number_density_interp.append(interp1d(energy_density, number_density)(energy_density_grid))

pressure_interp = np.array(pressure_interp)
number_density_interp = np.array(number_density_interp)

In [ ]:
prior_quantiles = []
astro_quantiles = []

for i in range(len(energy_density_grid)):
    prior_quantiles.append(np.quantile(pressure_interp[:,i], [0.05, 0.5, 0.95]))
    astro_quantiles.append(weighted_quantile(pressure_interp[:,i], [0.05, 0.5, 0.95], weights=astro_weights[:2000]))

prior_quantiles = np.array(prior_quantiles).T
astro_quantiles = np.array(astro_quantiles).T

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')

for p in pressure_interp:

    ax.plot(
        energy_density_grid, 
        p, 
        alpha=0.1, 
        c='C0'
        )

ax.plot(energy_density_grid, prior_quantiles[0], c='C1')
ax.plot(energy_density_grid, prior_quantiles[1], c='C1')
ax.plot(energy_density_grid, prior_quantiles[2], c='C1')

ax.plot(energy_density_grid, astro_quantiles[0], c='C3')
ax.plot(energy_density_grid, astro_quantiles[1], c='C3')
ax.plot(energy_density_grid, astro_quantiles[2], c='C3')

ax.set_ylim(1e-3, 1)
ax.set_xlim(0., 2)

ax.set_xlabel('Energy density [GeV/fm$^3$]')
_ = ax.set_ylabel('Pressure [GeV/fm$^3$]')

### Inspecting the macro draws

In [ ]:
fig, ax = plt.subplots()

for eos in collated_eos.eos[:5]:

    eos = int(eos)

    df = pd.read_csv(
        f'../data/eos-draws-default/mrgagn/DRAWmod1000-{int(eos/1000):06}/macro-draw-{eos:06}.csv'
    )

    mass = df.M
    radius = df.R

    print(np.max(mass))

    ax.plot(
        radius, 
        mass, 
        alpha=1, 
        c='C0'
        )

# ax.set_xlim(6, 18)
# ax.set_ylim(0.8, 2.2)

ax.set_xlabel('Radius, $R$ [km]')
_ = ax.set_ylabel('Mass, $M$ [$M_\odot$]')

In [ ]:
fig, ax = plt.subplots()

max_mass_list = []

for eos in collated_eos.eos[100:105]:

    eos = int(eos)

    df = pd.read_csv(f'{target_dir}/macro-draw-{eos:06}.csv')

    mass = df.M
    radius = df.R

    turning_index = np.argmax(mass)
    # mass = mass[:turning_index]
    # radius = radius[:turning_index]

    max_mass_list.append(np.max(mass))

    ax.plot(
        radius, 
        mass, 
        alpha=1, 
        c='C0'
        )

# ax.set_xlim(6, 18)
# ax.set_ylim(0.8, 2.2)

ax.set_xlabel('Radius, $R$ [km]')
_ = ax.set_ylabel('Mass, $M$ [$M_\odot$]')

In [ ]:
min(max_mass_list)

In [ ]:
# Load the full EoS data from the GP draws, and interpolate onto a consistent
# mass grid

mass_grid = np.linspace(0.8, 1.8, 1000)
radius_interp = []

for eos in collated_eos.eos[:2000]:

    eos = int(eos)

    df = pd.read_csv(f'{target_dir}/macro-draw-{eos:06}.csv')

    radius = df.R.values
    mass = df.M.values

    turning_index = np.argmax(mass)
    mass = mass[:turning_index]
    radius = radius[:turning_index]

    try:
        radius_interp.append(interp1d(mass, radius)(mass_grid))
    except:
        pass

radius_interp = np.array(radius_interp)

# Compute the weighted quantiles of the radius at each mass, using the 
# pre-computed weights
weighted_quantiles = []

for i in range(len(mass_grid)):
    weighted_quantiles.append(weighted_quantile(radius_interp[:,i], [0.05, 0.5, 0.95], weights=astro_weights[:2000]))

weighted_quantiles = np.array(weighted_quantiles).T

In [ ]:
len(radius_interp)

In [ ]:
fig, ax = plt.subplots()

for r in radius_interp:

    ax.plot(
        r, 
        mass_grid, 
        alpha=0.1, 
        c='C0'
        )
    
ax.plot(weighted_quantiles[0], mass_grid, c='C1')
ax.plot(weighted_quantiles[1], mass_grid, c='C1')
ax.plot(weighted_quantiles[2], mass_grid, c='C1')

ax.set_xlim(6, 18)
ax.set_ylim(0.8, 1.8)

ax.set_xlabel('Radius, $R$ [km]')
_ = ax.set_ylabel('Mass, $M$ [$M_\odot$]')